In TensorFlow 2.0, eager execution is turned on by default. The user interface is intuitive and flexible (running one-off operations is much easier and faster), but this can come at the expense of performance and deployability.

To get peak performance and to make your model deployable anywhere, use tf.function to make graphs out of your programs. Thanks to AutoGraph, a surprising amount of Python code just works with tf.function, but there are still pitfalls to be wary of.

The main takeaways and recommendations are:

- Don't rely on Python side effects like object mutation or list appends.

- tf.function works best with TensorFlow ops, rather than NumPy ops or Python primitives.

- When in doubt, use the for x in y idiom.

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

In [2]:
import traceback
import contextlib

# Some helper code to demonstrate the kinds of errors you might encounter.
@contextlib.contextmanager
def assert_raises(error_class):
    try:
        yield
    except error_class as e:
        print('Caught expected exception \n  {}:'.format(error_class))
        traceback.print_exc(limit=2)
    except Exception as e:
        raise e
    else:
        raise Exception('Expected {} to be raised but no error was raised!'.format(
        error_class))

# Basics
A tf.function you define is just like a core TensorFlow operation: You can execute it eagerly; you can use it in a graph; it has gradients; and so on.

In [3]:
@tf.function
def add(a,b):
    return a + b

add(tf.ones([2, 2]), tf.ones([2, 2]))  #  [[2., 2.], [2., 2.]]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 2.],
       [2., 2.]], dtype=float32)>

In [4]:
v = tf.Variable(1.)
with tf.GradientTape() as tape:
    result = add(v, 1.0)
tape.gradient(result, v)

<tf.Tensor: shape=(), dtype=float32, numpy=1.0>

In [5]:
#You can use functions inside functions

#tf.function
def dense_layer(x, w, b):
    return add(tf.matmul(x,w), b)
dense_layer(tf.ones([3,2]), tf.ones([2, 2]), tf.ones([2]))

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[3., 3.],
       [3., 3.],
       [3., 3.]], dtype=float32)>

# Tracing and polymorphism
Python's dynamic typing means that you can call functions with a variety of argument types, and Python will do something different in each scenario.

On the other hand, TensorFlow graphs require static dtypes and shape dimensions. tf.function bridges this gap by retracing the function when necessary to generate the correct graphs. Most of the subtlety of tf.function usage stems from this retracing behavior.

You can call a function with arguments of different types to see what is happening.

In [6]:
# Functions are polymorphic

@tf.function
def double(a):
    print("Tracing with", a)
    return a + a

print(double(tf.constant(1)))
print()
print(double(tf.constant(1.1)))
print()
print(double(tf.constant("a")))
print()

Tracing with Tensor("a:0", shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)

Tracing with Tensor("a:0", shape=(), dtype=float32)
tf.Tensor(2.2, shape=(), dtype=float32)

Tracing with Tensor("a:0", shape=(), dtype=string)
tf.Tensor(b'aa', shape=(), dtype=string)



To control the tracing behavior, use the following techniques:

- Create a new tf.function. Separate tf.function objects are guaranteed not to share traces.

- Use the get_concrete_function method to get a specific trace

- Specify input_signature when calling tf.function to trace only once per calling graph.

In [8]:
print("Obtaining concrete trace")
double_strings = double.get_concrete_function(tf.TensorSpec(shape=None, dtype=tf.string))
print("Executing traced function")
print(double_strings(tf.constant('a')))
print(double_strings(a=tf.constant('b')))
print("Using a concrete trace with incompatible types will throw an error")
with assert_raises(tf.errors.InvalidArgumentError):
    double_strings(tf.constant(1))

Obtaining concrete trace
Executing traced function
tf.Tensor(b'aa', shape=(), dtype=string)
tf.Tensor(b'bb', shape=(), dtype=string)
Using a concrete trace with incompatible types will throw an error
Caught expected exception 
  <class 'tensorflow.python.framework.errors_impl.InvalidArgumentError'>:


Traceback (most recent call last):
  File "<ipython-input-2-e40481c6c411>", line 8, in assert_raises
    yield
  File "<ipython-input-8-7d7f1df5239f>", line 8, in <module>
    double_strings(tf.constant(1))
tensorflow.python.framework.errors_impl.InvalidArgumentError: cannot compute __inference_double_79 as input #0(zero-based) was expected to be a string tensor but is a int32 tensor [Op:__inference_double_79]


In [9]:
@tf.function(input_signature=(tf.TensorSpec(shape=[None], dtype=tf.int32),))
def next_collatz(x):
    print("Tracing with", x)
    return tf.where(x % 2 == 0, x // 2, 3 * x + 1)

print(next_collatz(tf.constant([1, 2])))
# We specified a 1-D tensor in the input signature, so this should fail.
with assert_raises(ValueError):
    next_collatz(tf.constant([[1, 2], [3,4]]))

Tracing with Tensor("x:0", shape=(None,), dtype=int32)
tf.Tensor([4 1], shape=(2,), dtype=int32)
Caught expected exception 
  <class 'ValueError'>:


Traceback (most recent call last):
  File "<ipython-input-2-e40481c6c411>", line 8, in assert_raises
    yield
  File "<ipython-input-9-c0fb9efa7e9b>", line 9, in <module>
    next_collatz(tf.constant([[1, 2], [3,4]]))
ValueError: Python inputs incompatible with input_signature:
  inputs: (
    tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32))
  input_signature: (
    TensorSpec(shape=(None,), dtype=tf.int32, name=None))


# When to retrace?
A polymorphic tf.function keeps a cache of concrete functions generated by tracing. The cache keys are effectively tuples of keys generated from the function args and kwargs. The key generated for a tf.Tensor argument is its number of dimensions and type. The key generated for a Python primitive is its value. For all other Python types, the keys are based on the object id() so that methods are traced independently for each instance of a class. In the future, TensorFlow may add more sophisticated cachi ng for Python objects that can be safely converted to tensors.

See Concrete functions

## Python or Tensor args?
Often, Python arguments are used to control hyperparameters and graph constructions - for example, num_layers=10 or training=True or nonlinearity='relu'. So if the Python argument changes, it makes sense that you'd have to retrace the graph.

However, it's possible that a Python argument is not being used to control graph construction. In these cases, a change in the Python value can trigger needless retracing. Take, for example, this training loop, which AutoGraph will dynamically unroll. Despite the multiple traces, the generated graph is actually identical, so this is a bit inefficient.

In [12]:
def train_one_step():
    pass

@tf.function
def train(num_steps):
    print("Tracing with num_steps = {}".format(num_steps))
    for _ in tf.range(num_steps):
        train_one_step()

train(num_steps=10)
train(num_steps=20)

Tracing with num_steps = 10
Tracing with num_steps = 20


In [13]:
#The simple workaround here is to cast your arguments to Tensors if they do not affect the shape of the generated graph.
train(num_steps=tf.constant(10))
train(num_steps=tf.constant(20))

Tracing with num_steps = Tensor("num_steps:0", shape=(), dtype=int32)


# Side effects in tf.function
In general, Python side effects (like printing or mutating objects) only happen during tracing. So how can you reliably trigger side effects from tf.function?

The general rule of thumb is to only use Python side effects to debug your traces. Otherwise, TensorFlow ops like tf.Variable.assign, tf.print, and tf.summary are the best way to ensure your code will be traced and executed by the TensorFlow runtime with each call. In general using a functional style will yield the best results.

In [15]:
@tf.function
def f(x):
    print('Traced with', x)
    tf.print("Executed with", x)
f(1)
f(1)
f(30)

Traced with 1
Executed with 1
Executed with 1
Traced with 30
Executed with 30


If you would like to execute Python code during each invocation of a tf.function, tf.py_function is an exit hatch. The drawback of tf.py_function is that it's not portable or particularly performant, nor does it work well in distributed (multi-GPU, TPU) setups. Also, since tf.py_function has to be wired into the graph for differentiability, it casts all inputs/outputs to tensors.

In [17]:
external_list = []

def side_effect(x):
    print('Python side effect')
    external_list.append(x)
    
@tf.function
def f(x):
    tf.py_function(side_effect,inp=[x], Tout=[])

f(1)
f(1)
f(1)
assert len(external_list) == 3
# .numpy() call required because py_function casts 1 to tf.constant(1)
assert external_list[0].numpy() == 1


Python side effect
Python side effect
Python side effect


# Beware of Python state
Many Python features, such as generators and iterators, rely on the Python runtime to keep track of state. In general, while these constructs work as expected in Eager mode, many unexpected things can happen inside a tf.function due to tracing behavior.

To give one example, advancing iterator state is a Python side effect and therefore only happens during tracing.

In [18]:
external_var = tf.Variable(0)
@tf.function
def buggy_consume_next(iterator):
    external_var.assign_add(next(iterator))
    tf.print("Value of external_var:", external_var)
    
iterator = iter([0, 1, 2, 3])
buggy_consume_next(iterator)
# This reuses the first value from the iterator, rather than consuming the next value.
buggy_consume_next(iterator)
buggy_consume_next(iterator)

Value of external_var: 0
Value of external_var: 0
Value of external_var: 0


If an iterator is generated and consumed entirely within the tf.function, then it should work correctly. However, the entire iterator is probably being traced, which can lead to a giant graph. This may be what you want. But if you're training on an large in-memory dataset represented as a Python list, then this can generate a very large graph, and tf.function is unlikely to yield a speedup.

If you want to iterate over Python data, the safest way is to wrap it in a tf.data.Dataset and use the for x in y idiom. AutoGraph has special support for safely converting for loops when y is a tensor or tf.data.Dataset.

In [20]:
def measure_graph_size(f, *args):
    g = f.get_concrete_function(*args).graph
    print("{}({}) contains {} nodes in its graph".format(
    f.__name__, ', '.join(map(str,args)), len(g.as_graph_def().node)))
    
@tf.function
def train(dataset):
    loss = tf.constant(0)
    for x,y in dataset:
        loss += tf.abs(y - x)#some dummy computation
    return loss

small_data = [(1, 1)] * 2
big_data = [(1, 1)] * 10
measure_graph_size(train, small_data)
measure_graph_size(train, big_data)

measure_graph_size(train, tf.data.Dataset.from_generator(
                    lambda: small_data, (tf.int32, tf.int32)))

measure_graph_size(train, tf.data.Dataset.from_generator(
                    lambda: big_data, (tf.int32, tf.int32)))

train([(1, 1), (1, 1)]) contains 8 nodes in its graph
train([(1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1)]) contains 32 nodes in its graph
train(<FlatMapDataset shapes: (<unknown>, <unknown>), types: (tf.int32, tf.int32)>) contains 9 nodes in its graph
train(<FlatMapDataset shapes: (<unknown>, <unknown>), types: (tf.int32, tf.int32)>) contains 9 nodes in its graph


When wrapping Python/Numpy data in a Dataset, be mindful of tf.data.Dataset.from_generator versus tf.data.Dataset.from_tensors. The former will keep the data in Python and fetch it via tf.py_function which can have performance implications, whereas the latter will bundle a copy of the data as one large tf.constant() node in the graph, which can have memory implications.

Reading data from files via TFRecordDataset/CsvDataset/etc. is the most effective way to consume data, as then TensorFlow itself can manage the asynchronous loading and prefetching of data, without having to involve Python.

# Automatic Control Dependencies
A very appealing property of functions as the programming model, over a general dataflow graph, is that functions can give the runtime more information about what was the intended behavior of the code.

For example, when writing code which has multiple reads and writes to the same variables, a dataflow graph might not naturally encode the originally intended order of operations. In tf.function, we resolve ambiguities in execution order by referring to the execution order of statements in the original Python code. This way, ordering of stateful operations in a tf.function replicates the semantics of Eager mode.

This means there's no need to add manual control dependencies; tf.function is smart enough to add the minimal set of necessary and sufficient control dependencies for your code to run correctly.

In [23]:
#Automatic control dependencies
a = tf.Variable(1.0)
b = tf.Variable(2.0)

@tf.function
def f(x,y):
    a.assign(y * b)
    b.assign_add(x * a)
    return a + b

f(1.0, 2.0)

<tf.Tensor: shape=(), dtype=float32, numpy=10.0>

# Variables
We can use the same idea of leveraging the intended execution order of the code to make variable creation and utilization very easy in tf.function. There is one very important caveat, though, which is that with variables it's possible to write code which behaves differently in eager mode and graph mode.

Specifically, this will happen when you create a new Variable with each call. Due to tracing semantics, tf.function will reuse the same variable each call, but eager mode will create a new variable with each call. To guard against this mistake, tf.function will raise an error if it detects dangerous variable creation behavior.

In [24]:
@tf.function
def f(x):
    v = tf.Variable(1.0)
    v.assign_add(x)
    return x
with assert_raises(ValueError):
    f(1.0)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Caught expected exception 
  <class 'ValueError'>:


Traceback (most recent call last):
  File "<ipython-input-2-e40481c6c411>", line 8, in assert_raises
    yield
  File "<ipython-input-24-0e47770510e0>", line 7, in <module>
    f(1.0)
ValueError: in converted code:

    <ipython-input-24-0e47770510e0>:3 f  *
        v = tf.Variable(1.0)
    /anaconda3/lib/python3.7/site-packages/tensorflow_core/python/ops/variables.py:260 __call__
        return cls._variable_v2_call(*args, **kwargs)
    /anaconda3/lib/python3.7/site-packages/tensorflow_core/python/ops/variables.py:254 _variable_v2_call
        shape=shape)
    /anaconda3/lib/python3.7/site-packages/tensorflow_core/python/ops/variables.py:65 getter
        return captured_getter(captured_previous, **kwargs)
    /anaconda3/lib/python3.7/site-packages/tensorflow_core/python/eager/def_function.py:502 invalid_creator_scope
        "tf.function-decorated function tried to create "

    ValueError: tf.function-decorated function tried to create variables on non-first call.



In [25]:
#Non-ambiguous code is ok, though.

v = tf.Variable(1.0)

@tf.function
def f(x):
    return v.assign_add(x)

print(f(1.0))
print(f(2.0))

tf.Tensor(2.0, shape=(), dtype=float32)
tf.Tensor(4.0, shape=(), dtype=float32)


You can also create variables inside a tf.function as long as we can prove that those variables are created only the first time the function is executed.



In [26]:
class C:
    pass

obj = C()
obj.v = None

@tf.function
def g(x):
    if obj.v is None:
        obj.v = tf.Variable(1.0)
    return obj.v.assign_add(x)

print(g(1.0))
print(g(2.0))

tf.Tensor(2.0, shape=(), dtype=float32)
tf.Tensor(4.0, shape=(), dtype=float32)


Variable initializers can depend on function arguments and on values of other variables. We can figure out the right initialization order using the same method we use to generate control dependencies.

In [27]:
state = []
@tf.function
def fn(x):
    if not state:
        state.append(tf.Variable(2.0 *x))
        state.append(tf.Variable(state[0] * 3.0))
    return state[0] * x * state[1]

print(fn(tf.constant(1.0)))
print(fn(tf.constant(3.0)))

tf.Tensor(12.0, shape=(), dtype=float32)
tf.Tensor(36.0, shape=(), dtype=float32)


# Using AutoGraph
The autograph library is fully integrated with tf.function, and it will rewrite conditionals and loops which depend on Tensors to run dynamically in the graph.

tf.cond and tf.while_loop continue to work with tf.function, but code with control flow is often easier to write and understand when written in imperative style.

In [28]:
#If you're curious you can inspect the code autograph generates.

def f(x):
    while tf.reduce_sum(x) > 1:
        tf.print(x)
        x = tf.tanh(x)
        
        return x
print(tf.autograph.to_code(f))

def tf__f(x):
  do_return = False
  retval_ = ag__.UndefinedReturnValue()
  with ag__.FunctionScope('f', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:

    def get_state():
      return ()

    def set_state(_):
      pass

    def loop_body(x, retval_, do_return):
      ag__.converted_call(tf.print, (x,), None, fscope)
      x = ag__.converted_call(tf.tanh, (x,), None, fscope)
      do_return = True
      retval_ = fscope.mark_return_value(x)
      return x, retval_, do_return

    def loop_test(x, retval_, do_return):
      return ag__.and_(lambda : ag__.not_(do_return), lambda : ag__.converted_call(tf.reduce_sum, (x,), None, fscope) > 1)
    x, retval_, do_return = ag__.while_stmt(loop_test, loop_body, get_state, set_state, (x, retval_, do_return), ('x', 'retval_', 'do_return'), ())
  do_return,
  return ag__.retval(retval_)



# AutoGraph: Conditionals
AutoGraph will convert if statements into the equivalent tf.cond calls.

This substitution is made if the condition is a Tensor. Otherwise, the conditional is executed during tracing.

Here is a function that checks if the resulting graph uses tf.cond:

In [29]:
def test_tf_cond(f, *args):
    g = f.get_concrete_function(*args).graph
    if any(node.name == 'cond' for node in g.as_graph_def().node):
        print('{}({}) uses tf.cond.'.format(
        f.__name__, ', '.join(map(str, args))))
        
    else:
        print("{}({}) executes normally.".format(
        f.__name__, ', '.join(map(str, args))))
    print("  result: ",f(*args).numpy())

In [30]:
# This substitution is made if the condition is a Tensor. Otherwise, the conditional is executed during tracing.

# Passing a python True executes the conditional normally:
@tf.function
def dropout(x, training=True):
    if training:
        x = tf.nn.dropout(x, rate=0.5)
    return x

In [31]:
test_tf_cond(dropout, tf.ones([10], dtype=tf.float32), True)

dropout(tf.Tensor([1. 1. 1. 1. 1. 1. 1. 1. 1. 1.], shape=(10,), dtype=float32), True) executes normally.
  result:  [2. 2. 0. 0. 2. 0. 0. 0. 2. 2.]


In [32]:
#But passing a tensor replaces the python if with a tf.cond:
test_tf_cond(dropout, tf.ones([10], dtype=tf.float32), tf.constant(True))   

dropout(tf.Tensor([1. 1. 1. 1. 1. 1. 1. 1. 1. 1.], shape=(10,), dtype=float32), tf.Tensor(True, shape=(), dtype=bool)) uses tf.cond.
  result:  [0. 2. 0. 2. 0. 2. 2. 0. 2. 0.]


tf.cond has a number of subtleties.

it works by tracing both sides of the conditional, and then choosing the appropriate branch at runtime, depending on the condition. Tracing both sides can result in unexpected execution of Python code.

In [34]:
@tf.function
def f(x):
    if x > 0:
        x = x + 1.
        print("Tracing `then` branch")
    else:
        x = x - 1.
        print("Tracing `else` branch")
    return x

f(-1.0).numpy()

Tracing `else` branch


-2.0

In [35]:
f(1.0).numpy()

Tracing `then` branch


2.0

In [36]:
f(tf.constant(1.0)).numpy()

Tracing `then` branch
Tracing `else` branch


2.0

In [38]:
#It requires that if one branch creates a tensor used downstream, the other branch must also create that tensor.

@tf.function
def f():
    if tf.constant(True):
        x = tf.ones([3, 3])
    return x

#Throws an error because both branches need to define 'x'
with assert_raises(ValueError):
    f()

Caught expected exception 
  <class 'ValueError'>:


Traceback (most recent call last):
  File "<ipython-input-2-e40481c6c411>", line 8, in assert_raises
    yield
  File "<ipython-input-38-83dfc013b139>", line 11, in <module>
    f()
ValueError: in converted code:

    <ipython-input-38-83dfc013b139>:5 f  *
        if tf.constant(True):
    /anaconda3/lib/python3.7/site-packages/tensorflow_core/python/autograph/operators/control_flow.py:918 if_stmt
        basic_symbol_names, composite_symbol_names)
    /anaconda3/lib/python3.7/site-packages/tensorflow_core/python/autograph/operators/control_flow.py:956 tf_if_stmt
        error_checking_orelse)
    /anaconda3/lib/python3.7/site-packages/tensorflow_core/python/util/deprecation.py:507 new_func
        return func(*args, **kwargs)
    /anaconda3/lib/python3.7/site-packages/tensorflow_core/python/ops/control_flow_ops.py:1174 cond
        return cond_v2.cond_v2(pred, true_fn, false_fn, name)
    /anaconda3/lib/python3.7/site-packages/tensorflow_core/python/ops/cond_v2.py:90 cond_v2
        o

If you want to be sure that a particular section of control flow is never converted by autograph, then explicitly convert the object to a python type so an error is raised instead:



In [39]:
@tf.function
def f(x,y):
    if bool(x):
        y = y + 1
        print("Tracing `then` branch")
    else:
        y = y - 1
        print("Tracing `else` branch")
    return y

In [40]:
f(True, 0).numpy()

Tracing `then` branch


1

In [41]:
f(False, 0).numpy()

Tracing `else` branch


-1

In [42]:
with assert_raises(TypeError):
    f(tf.constant(True), 0.0)

Caught expected exception 
  <class 'TypeError'>:


Traceback (most recent call last):
  File "<ipython-input-2-e40481c6c411>", line 8, in assert_raises
    yield
  File "<ipython-input-42-ec9b920ba5e6>", line 2, in <module>
    f(tf.constant(True), 0.0)
tensorflow.python.framework.errors_impl.OperatorNotAllowedInGraphError: in converted code:

    <ipython-input-39-9bf9e8af3b58>:3 f  *
        if bool(x):
    /anaconda3/lib/python3.7/site-packages/tensorflow_core/python/autograph/impl/api.py:416 converted_call
        return py_builtins.overload_of(f)(*args)
    /anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py:757 __bool__
        self._disallow_bool_casting()
    /anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py:523 _disallow_bool_casting
        "using a `tf.Tensor` as a Python `bool`")
    /anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py:510 _disallow_when_autograph_enabled
        " decorating it directly with @tf.function.".format(task))

    

# AutoGraph and loops
AutoGraph has a few simple rules for converting loops.

- for: Convert if the iterable is a tensor
- while: Convert if the while condition depends on a tensor

If a loop is converted, it will be dynamically unrolled with tf.while_loop, or in the special case of a for x in tf.data.Dataset, transformed into tf.data.Dataset.reduce.

If a loop is not converted, it will be statically unrolled

In [43]:
def test_dynamically_unrolled(f, *args):
    g = f.get_concrete_function(*args).graph
    if any(node.name == 'while' for node in g.as_graph_def().node):
        print("{}({}) uses tf.while_loop.".format(
        f.__name__, ', '.join(map(str, args))))
    elif any(node.name == 'ReduceDataset' for node in g.as_graph_def().node):
        print("{}({}) uses tf.data.Dataset.reduce.".format(
        f.__name__, ', '.join(map(str, args))))
    else:
        print("{}({}) gets unrolled.".format(
        f.__name__, ', '.join(map(str, args))))

# For loops
Here is a tf.function that demonstrates static unrolling:

In [44]:
@tf.function
def for_in_range():
    x = 0
    for i in range(5):
        x += 1
    return x

test_dynamically_unrolled(for_in_range)

for_in_range() gets unrolled.


In [46]:
@tf.function
def for_in_tfrange():
    x = tf.constant(0, dtype=tf.int32)
    for i in tf.range(5):
        x += i
    return x
test_dynamically_unrolled(for_in_tfrange)

for_in_tfrange() uses tf.while_loop.


In [47]:
@tf.function
def for_in_tfdataset():
    x = tf.constant(0, dtype=tf.int64)
    for i in tf.data.Dataset.range(5):
        x += i
    return x


test_dynamically_unrolled(for_in_tfdataset)

for_in_tfdataset() uses tf.data.Dataset.reduce.


In [48]:
@tf.function
def while_py_cond():
    x = 5
    while x > 0:
        x -= 1
    return x

test_dynamically_unrolled(while_py_cond)

while_py_cond() gets unrolled.


If you have a break or early return clause that depends on a tensor, the top-level condition or iterable should also be a tensor.

Compare the following examples:

In [49]:
@tf.function
def while_py_true_py_break(x):
    while True: #py true
        if x == 0: # py break
            break
        x -= 1
        return x
    
test_dynamically_unrolled(while_py_true_py_break, 5)

while_py_true_py_break(5) gets unrolled.


In [50]:
@tf.function
def buggy_while_py_true_tf_break(x):
    while True: # py true
        if tf.equal(x,0): #tf break
            break
        x -= 1
    return x
with assert_raises(TypeError):
    test_dynamically_unrolled(buggy_while_py_true_tf_break, 5)

Caught expected exception 
  <class 'TypeError'>:


Traceback (most recent call last):
  File "<ipython-input-2-e40481c6c411>", line 8, in assert_raises
    yield
  File "<ipython-input-50-bfc569872864>", line 9, in <module>
    test_dynamically_unrolled(buggy_while_py_true_tf_break, 5)
tensorflow.python.framework.errors_impl.OperatorNotAllowedInGraphError: in converted code:

    <ipython-input-50-bfc569872864>:3 buggy_while_py_true_tf_break  *
        while True: # py true
    /anaconda3/lib/python3.7/site-packages/tensorflow_core/python/autograph/operators/control_flow.py:755 while_stmt
        return _py_while_stmt(test, body, get_state, set_state, init_vars, opts)
    /anaconda3/lib/python3.7/site-packages/tensorflow_core/python/autograph/operators/control_flow.py:870 _py_while_stmt
        while test(*loop_vars):
    /anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py:757 __bool__
        self._disallow_bool_casting()
    /anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py:523 

In [51]:
@tf.function
def while_tf_true_tf_break(x):
    while tf.constant(True): #tf true
        if x == 0: # py break
            break
        x -= 1
    return x

test_dynamically_unrolled(while_tf_true_tf_break, 5)

while_tf_true_tf_break(5) uses tf.while_loop.


In [52]:
@tf.function
def buggy_py_for_tf_break():
    x = 0
    for i in range(5): # py for
        if tf.equal(i,3): #tf break
            break
        x += i
    return x
with assert_raises(TypeError):
    test_dynamically_unrolled(buggy_py_for_tf_break)

Caught expected exception 
  <class 'TypeError'>:


Traceback (most recent call last):
  File "<ipython-input-2-e40481c6c411>", line 8, in assert_raises
    yield
  File "<ipython-input-52-52ec0848391b>", line 10, in <module>
    test_dynamically_unrolled(buggy_py_for_tf_break)
tensorflow.python.framework.errors_impl.OperatorNotAllowedInGraphError: in converted code:

    <ipython-input-52-52ec0848391b>:4 buggy_py_for_tf_break  *
        for i in range(5): # py for
    /anaconda3/lib/python3.7/site-packages/tensorflow_core/python/autograph/operators/control_flow.py:339 for_stmt
        return _py_for_stmt(iter_, extra_test, body, get_state, set_state, init_vars)
    /anaconda3/lib/python3.7/site-packages/tensorflow_core/python/autograph/operators/control_flow.py:348 _py_for_stmt
        if extra_test is not None and not extra_test(*state):
    /anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py:757 __bool__
        self._disallow_bool_casting()
    /anaconda3/lib/python3.7/site-packages/tensorflow_core/python/

In [53]:
@tf.function
def tf_for_py_break():
    x = 0
    for i in tf.range(5): # tf for
        if i == 3: # py break
            break
        x += i
    return x

test_dynamically_unrolled(tf_for_py_break)

tf_for_py_break() uses tf.while_loop.


In order to accumulate results from a dynamically unrolled loop, you'll want to use tf.TensorArray.

In [54]:
batch_size = 2
seq_len = 3
feature_size = 4

def rnn_step(inp, state):
    return inp + state

@tf.function
def dynamic_rnn(rnn_step, input_data, initial_state):
    # [batch, time, features] -> [time, batch, features]
    input_data = tf.transpose(input_data, [1, 0, 2])
    max_seq_len = input_data.shape[0]
    
    
    states = tf.TensorArray(tf.float32, size=max_seq_len)
    state = initial_state
    for i in tf.range(max_seq_len):
        state = rnn_step(input_data[i], state)
        states = states.write(i, state)
    return tf.transpose(states.stack(), [1, 0, 2])

dynamic_rnn(rnn_step,
           tf.random.uniform([batch_size, seq_len, feature_size]),
           tf.zeros([batch_size, feature_size]))


<tf.Tensor: shape=(2, 3, 4), dtype=float32, numpy=
array([[[0.9382771 , 0.84717345, 0.35492063, 0.78523815],
        [1.5913814 , 1.3970994 , 1.3392398 , 1.5786211 ],
        [2.584334  , 1.6167775 , 2.1044822 , 2.1445684 ]],

       [[0.49491906, 0.63566387, 0.14135838, 0.03258014],
        [0.6510335 , 0.8688539 , 0.23932862, 0.58064735],
        [1.5056176 , 1.3179424 , 0.24899626, 0.8400575 ]]], dtype=float32)>

# Gotcha's
As with tf.cond, tf.while_loop also comes with a number of subtleties.

#### Zero iterations
Since a loop can execute 0 times, all tensors used downstream of the while_loop must be initialized above the loop.

Here is an example of incorrect code:

In [56]:
@tf.function
def buggy_loop_var_uninitialized():
    for i in tf.range(3):
        x = i
    return x

with assert_raises(ValueError):
    buggy_loop_var_uninitialized()

Caught expected exception 
  <class 'ValueError'>:


Traceback (most recent call last):
  File "<ipython-input-2-e40481c6c411>", line 8, in assert_raises
    yield
  File "<ipython-input-56-fb9c665fb220>", line 8, in <module>
    buggy_loop_var_uninitialized()
ValueError: in converted code:

    <ipython-input-56-fb9c665fb220>:3 buggy_loop_var_uninitialized  *
        for i in tf.range(3):
    /anaconda3/lib/python3.7/site-packages/tensorflow_core/python/autograph/operators/control_flow.py:315 for_stmt
        composite_symbol_names)
    /anaconda3/lib/python3.7/site-packages/tensorflow_core/python/autograph/operators/control_flow.py:419 _tf_range_for_stmt
        _disallow_undefs_into_loop(*init_vars)
    /anaconda3/lib/python3.7/site-packages/tensorflow_core/python/autograph/operators/control_flow.py:97 _disallow_undefs_into_loop
        ' before the loop: {}'.format(tuple(s.symbol_name for s in undefined)))

    ValueError: TensorFlow requires that the following symbols must be defined before the loop: ('x',)



And the correct version:



In [57]:
@tf.function
def f():
    x = tf.constant(0)
    for i in tf.range(3):
        x = i
    return x

f()

<tf.Tensor: shape=(), dtype=int32, numpy=2>

# Consistent shapes and types
The shape/dtypes of all loop variables must stay consistent with each iteration.

Here is an incorrect example that attempts to change a tensor's type:

In [58]:
@tf.function
def buggy_loop_type_changes():
    x = tf.constant(0, dtype=tf.float32)
    for i in tf.range(3): # Yields tensors of tf.int32
        x = i
    return x

with assert_raises(TypeError):
    buggy_loop_type_changes()

Caught expected exception 
  <class 'TypeError'>:


Traceback (most recent call last):
  File "<ipython-input-2-e40481c6c411>", line 8, in assert_raises
    yield
  File "<ipython-input-58-ce2be3d5832c>", line 9, in <module>
    buggy_loop_type_changes()
TypeError: in converted code:

    <ipython-input-58-ce2be3d5832c>:4 buggy_loop_type_changes  *
        for i in tf.range(3): # Yields tensors of tf.int32
    /anaconda3/lib/python3.7/site-packages/tensorflow_core/python/autograph/operators/control_flow.py:315 for_stmt
        composite_symbol_names)
    /anaconda3/lib/python3.7/site-packages/tensorflow_core/python/autograph/operators/control_flow.py:478 _tf_range_for_stmt
        opts=opts,
    /anaconda3/lib/python3.7/site-packages/tensorflow_core/python/autograph/operators/control_flow.py:794 _tf_while_stmt
        aug_init_vars, **opts)
    /anaconda3/lib/python3.7/site-packages/tensorflow_core/python/ops/control_flow_ops.py:2675 while_loop
        back_prop=back_prop)
    /anaconda3/lib/python3.7/site-packages/tensorflow_core/pytho

In [59]:
#Here is an incorrect example that attempts to change a Tensor's shape while iterating:

@tf.function
def buggy_concat():
    x = tf.ones([0, 10])
    for i in tf.range(5):
        x = tf.concat([x, tf.ones([1, 10])], axis=0)
    return x

with assert_raises(ValueError):
    buggy_concat()

Caught expected exception 
  <class 'ValueError'>:


Traceback (most recent call last):
  File "<ipython-input-2-e40481c6c411>", line 8, in assert_raises
    yield
  File "<ipython-input-59-3aae619d68a3>", line 11, in <module>
    buggy_concat()
ValueError: in converted code:

    <ipython-input-59-3aae619d68a3>:6 buggy_concat  *
        for i in tf.range(5):
    /anaconda3/lib/python3.7/site-packages/tensorflow_core/python/autograph/operators/control_flow.py:315 for_stmt
        composite_symbol_names)
    /anaconda3/lib/python3.7/site-packages/tensorflow_core/python/autograph/operators/control_flow.py:478 _tf_range_for_stmt
        opts=opts,
    /anaconda3/lib/python3.7/site-packages/tensorflow_core/python/autograph/operators/control_flow.py:794 _tf_while_stmt
        aug_init_vars, **opts)
    /anaconda3/lib/python3.7/site-packages/tensorflow_core/python/ops/control_flow_ops.py:2675 while_loop
        back_prop=back_prop)
    /anaconda3/lib/python3.7/site-packages/tensorflow_core/python/ops/while_v2.py:194 while_loop
        add_cont

In [60]:
@tf.function
def concat_with_padding():
    x = tf.zeros([5, 10])
    for i in tf.range(5):
        x = tf.concat([x[:i], tf.ones([1,10]), tf.zeros([4-i, 10])], axis=0)
        x.set_shape([5,10])
    return x

concat_with_padding()

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]], dtype=float32)>